### Ingest drivers.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [1]:
v_data_source = "" #Provided by Data Factory

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 3, Finished, Available)

In [2]:
%run configuration

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 5, Finished, Available)

In [3]:
%run common_functions

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 9, Finished, Available)

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 10, Finished, Available)

In [5]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)])

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 11, Finished, Available)

In [6]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)])

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 12, Finished, Available)

In [7]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json(f"{bronze_folder_path}/drivers.json")

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 13, Finished, Available)

##### Step 2 - Rename columns and add new columns
1. driverId renamed to driver_id  
1. driverRef renamed to driver_ref  
1. ingestion date added
1. name added with concatenation of forename and surname

In [8]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 14, Finished, Available)

In [14]:
drivers_with_columns_df = add_ingestion_date(drivers_df)
display(drivers_with_columns_df)

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, a2118d7e-2f1b-4896-b60f-88c1465bb740)

In [10]:
drivers_with_columns_df = drivers_with_columns_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("data_source", lit(v_data_source))

display(drivers_with_columns_df)

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, d56f1c8c-ee4a-4cfc-88c2-2db24004f461)

##### Step 3 - Drop the unwanted columns
1. name.forename
1. name.surname
1. url

In [11]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))
display(drivers_final_df)

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5b0aa12c-ae5a-4597-b030-73d83b38abf0)

##### Step 4 - Write to output to processed container in parquet format

In [12]:
drivers_final_df.write.mode('overwrite').parquet(f"{silver_folder_path}/drivers")

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 18, Finished, Available)

In [13]:
df_parquet = spark.read.parquet(f"{silver_folder_path}/drivers")
display(df_parquet)

StatementMeta(, 62f706d5-5a42-4513-ad60-3ecc133fe02c, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, ba18ff67-519a-4c4e-8627-0b73768d328e)